In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_vertical_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,25691.09,-0.09,0.03,-0.05,-0.03,-0.0,-0.00,0.00
3,-0.09,5998.59,-0.01,0.03,0.00,0.0,0.00,-0.00
4,0.03,-0.01,1183.24,-0.02,-0.00,0.0,-0.00,0.00
5,-0.05,0.03,-0.02,219.39,0.00,0.0,0.00,-0.00
6,-0.03,0.00,-0.00,0.00,36.37,-0.0,0.00,0.00
7,-0.00,0.00,0.00,0.00,-0.00,5.6,-0.00,-0.00
8,-0.00,0.00,-0.00,0.00,0.00,-0.0,0.87,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.0,0.00,0.12


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,0.00001,-0.00002,-0.00003,-0.00001,-0.00002,0.00000
3,-0.00001,1.00000,-0.00000,0.00002,0.00001,0.00002,0.00000,-0.00001
4,0.00001,-0.00000,1.00000,-0.00004,-0.00002,0.00004,-0.00001,0.00001
5,-0.00002,0.00002,-0.00004,1.00000,0.00002,0.00000,0.00001,-0.00002
6,-0.00003,0.00001,-0.00002,0.00002,1.00000,-0.00001,0.00002,0.00003
7,-0.00001,0.00002,0.00004,0.00000,-0.00001,1.00000,-0.00001,-0.00001
8,-0.00002,0.00000,-0.00001,0.00001,0.00002,-0.00001,1.00000,0.00001
9,0.00000,-0.00001,0.00001,-0.00002,0.00003,-0.00001,0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.1594771150668178

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[8.26047721e+07 4.40609253e+06 1.66723199e+05 5.50273658e+03
 1.41850203e+02 3.00027102e+00 5.41746790e-02 2.28944874e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999358,-0.035204,-0.006526,-0.001209,-0.000201,-0.000031,-0.000005,-6.512302e-07
1,0.034952,0.998778,-0.034397,-0.006092,-0.001009,-0.000155,-0.000024,-3.378265e-06
2,0.007667,0.033882,0.998632,-0.038576,-0.006139,-0.000941,-0.000146,-2.011105e-05
3,0.001705,0.007296,0.038034,0.998312,-0.042762,-0.006368,-0.000982,-1.367049e-04
4,0.000353,0.001507,0.007654,0.042087,0.997661,-0.052705,-0.007883,-1.091090e-03
5,0.000072,0.000307,0.001558,0.008418,0.051586,0.995456,-0.078871,-1.055605e-02
6,0.000017,0.000070,0.000356,0.001920,0.011589,0.076142,0.987349,-1.385885e-01
7,0.000005,0.000020,0.000100,0.000542,0.003265,0.021208,0.137327,9.902932e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006419236937313944,
 0.001222211553456809,
 0.0013676669317196355,
 0.0016878407167219,
 0.002339449731586285,
 0.004543661270174804,
 0.012651277420765772,
 0.009706813923219149]